In [26]:
# import the required libraries
import pandas as pd
import numpy as np
# read data from csv file
df = pd.read_csv('Training_CETC331.csv')
aps = pd.DataFrame(pd.read_excel('Information of pre-installed APs.xlsx'))


In [27]:
df.shape


(955, 60)

In [28]:
df.head()


,MAC1,MAC2,MAC3,MAC4,MAC5,MAC6,MAC7,MAC8,MAC9,MAC10,...,MAC51,MAC52,ECoord,NCoord,FloorID,BuildingID,SceneID,UserID,PhoneID,SampleTimes
0,-44,-48,-71,-85,-92,100,100,100,-88,100,...,100,100,45.0,17.5,1,1,1,4,3,1
1,-37,-50,-72,-85,-93,100,-93,100,-86,100,...,100,100,45.5,17.5,1,1,1,4,3,1
2,-41,-51,-68,-84,-88,100,-93,100,-84,100,...,100,100,46.0,17.5,1,1,1,4,3,1
3,-38,-50,-67,-89,-83,-90,-92,100,-83,100,...,100,100,46.5,17.5,1,1,1,4,3,1
4,-35,-57,-63,-87,-86,-87,100,100,-81,100,...,100,100,46.9,17.5,1,1,1,4,3,1


In [29]:
# print unique values in phoneid column
aps.head()

,ID,ECoord,NCoord,FloorID,Attribute_2.4,Frequency_2.4,Attribute_5,Frequency_5
0,1,48.152072,18.698414,1,MAC1,2412,MAC2,5745
1,2,52.482000,36.853200,1,MAC47,2412,MAC48,5745
2,3,45.007523,20.451347,1,MAC21,2412,MAC22,5745
3,4,53.320000,39.345300,1,MAC17,2412,MAC18,5745
4,5,52.538000,55.978100,1,MAC5,2412,MAC6,5745


In [30]:
arr = aps.to_numpy()
# arr


In [31]:
aps.iloc[8]['Attribute_2.4']


'MAC49'

In [32]:
# store coordinates of APs in a list
coOrdinates = {}
for index, d in aps.iterrows():
    # print(d)
    coOrdinates[d['Attribute_2.4']] = [d['ECoord'], d['NCoord']]
    coOrdinates[d['Attribute_5']] = [d['ECoord'], d['NCoord']]
# coOrdinates


In [33]:
rssi = []
for index, d in df.iterrows():
    rssi.append([d['MAC1'], d['MAC2'], d['MAC3'], d['MAC4'], d['MAC5'], d['MAC6'], d['MAC7'],
                 d['MAC8'], d['MAC9'], d['MAC10'], d['MAC11'], d['MAC12'], d['MAC13'], d['MAC14'], d['MAC15'],
                 d['MAC16'], d['MAC17'], d['MAC18'], d['MAC19'], d['MAC20'], d['MAC21'], d['MAC22'], d['MAC23'],
                 d['MAC24'], d['MAC25'], d['MAC26'], d['MAC27'], d['MAC28'], d['MAC29'], d['MAC30'], d['MAC31'],
                 d['MAC32'], d['MAC33'], d['MAC34'], d['MAC35'], d['MAC36'], d['MAC37'], d['MAC38'], d['MAC39'],
                 d['MAC40'], d['MAC41'], d['MAC42'], d['MAC43'], d['MAC44'], d['MAC45'], d['MAC46'], d['MAC47'],
                 d['MAC48'], d['MAC49'], d['MAC50'], d['MAC51'], d['MAC52']
                 ])
# rssi


In [34]:
pl = 4.0  # reference path loss coefficient,
alpha = 2.3  # path loss exponent


def distance(rssi: list,alpha):
    '''This method will calculate the distance of each access point from the device using the RSSI values'''
    d = []
    for val in rssi:
        x = 10**((pl-val)/(10*alpha))
        if val == 100.0:
            # print('yo')
            x = 0
        d.append(x)
    return d


In [36]:
x = rssi[0]
dist = distance(x,4)


In [37]:
def calcWeight(dist):
    '''This method will calculate the weight of each access point from the device using the distance values'''
    w = []
    tot = 0
    for x in dist:
        if x != 0:
            tot = tot+1/x
        # tot=tot+1/x
    for val in dist:
        if val != 0:
            x = 1/(val*tot)
        else:
            x = 0
        w.append(x)
    return w


In [38]:
weights = calcWeight(dist)
weights


[0.1990433805886937,
 0.15810577713657037,
 0.04206760033262385,
 0.01879090682974747,
 0.012558788285260263,
 0,
 0,
 0,
 0.015810577713657046,
 0,
 0.011193031841061298,
 0,
 0.01409119221973332,
 0,
 0,
 0,
 0.029781580314637818,
 0.015810577713657046,
 0,
 0,
 0.23656350118625666,
 0.12558788285260264,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.01879090682974747,
 0,
 0,
 0,
 0,
 0,
 0.013302943274874827,
 0,
 0.015810577713657046,
 0,
 0.01990433805886936,
 0,
 0.01990433805886936,
 0.01879090682974747,
 0.01409119221973332,
 0,
 0,
 0]

In [39]:
def calcLocation(weights, wifiLocs):
    '''This method will calculate the location of the device using the weights and the location of the access points'''
    x = 0
    y = 0
    for weight in weights:
        # print(wifiLocs['MAC'+str(weights.index(weight)+1)])
        xi = wifiLocs['MAC'+str(weights.index(weight)+1)][0]
        yi = wifiLocs['MAC'+str(weights.index(weight)+1)][1]
        x = x+xi*weight
        y = y+yi*weight
    return x, y


In [40]:
x, y = calcLocation(weights, coOrdinates)
x, y


(47.84761970854741, 21.198392520957718)

In [41]:
# dist between 2 points
def dist_(x1, y1, x2, y2):
    return np.sqrt((x2-x1)**2+(y2-y1)**2)
dist_(46.87827665756832, 20.058309975329635,45,17.5)

3.173778998644541

In [ ]:
df.head()

,MAC1,MAC2,MAC3,MAC4,MAC5,MAC6,MAC7,MAC8,MAC9,MAC10,...,MAC51,MAC52,ECoord,NCoord,FloorID,BuildingID,SceneID,UserID,PhoneID,SampleTimes
0,-44,-48,-71,-85,-92,100,100,100,-88,100,...,100,100,45.0,17.5,1,1,1,4,3,1
1,-37,-50,-72,-85,-93,100,-93,100,-86,100,...,100,100,45.5,17.5,1,1,1,4,3,1
2,-41,-51,-68,-84,-88,100,-93,100,-84,100,...,100,100,46.0,17.5,1,1,1,4,3,1
3,-38,-50,-67,-89,-83,-90,-92,100,-83,100,...,100,100,46.5,17.5,1,1,1,4,3,1
4,-35,-57,-63,-87,-86,-87,100,100,-81,100,...,100,100,46.9,17.5,1,1,1,4,3,1


In [53]:
def predictLocation():
    '''This method will predict the location of the device using the RSSI values'''
    mean=0
    min_error=10000000
    alpha=-1
    for al in np.arange(2,4.1,0.1):
        for i in range(len(rssi)):
            dist = distance(rssi[i],al)
            weights = calcWeight(dist)
            x, y = calcLocation(weights, coOrdinates)
            print("Actual Location\t\tPredicted Location\t\tError (in meters)")
                            # rounding off the predicted location to 2 decimal places
            x = round(x, 2)
            y = round(y, 2)
            xi=round(df.iloc[i]['ECoord'],2)
            yi=df.iloc[i]['NCoord'].round(2)
            print(xi,yi, '\t\t', x, y, '\t\t', dist_(x, y, xi,yi))
            mean=mean+dist_(x, y, xi,yi)
        if mean < min_error:
            min_error=mean
            alpha=al
    print("min Mean Error: ",min_error/len(rssi))
    print("recommended alpha: ",alpha)  

In [54]:
predictLocation()

Actual Location		Predicted Location		Error (in meters)
45.0 17.5 		 46.72 19.9 		 2.9526936854336907
Actual Location		Predicted Location		Error (in meters)
45.5 17.5 		 47.21 19.82 		 2.882099928871309
Actual Location		Predicted Location		Error (in meters)
46.0 17.5 		 47.6 19.7 		 2.720294101747089
Actual Location		Predicted Location		Error (in meters)
46.5 17.5 		 46.71 20.29 		 2.7978920636793685
Actual Location		Predicted Location		Error (in meters)
46.9 17.5 		 46.92 19.93 		 2.4300823031329615
Actual Location		Predicted Location		Error (in meters)
47.4 17.5 		 46.94 19.94 		 2.482982078066615
Actual Location		Predicted Location		Error (in meters)
45.0 18.0 		 46.2 20.61 		 2.8726468630863775
Actual Location		Predicted Location		Error (in meters)
45.5 18.0 		 46.11 20.35 		 2.427879733429975
Actual Location		Predicted Location		Error (in meters)
46.0 18.0 		 46.66 19.99 		 2.096592473515058
Actual Location		Predicted Location		Error (in meters)
46.5 18.0 		 46.97 19.95 		 2.005841

In [51]:
import numpy as np
from filterpy.kalman import KalmanFilter

def position_estimation(zs, x0, P0, F, H, Q, R):
    kf = KalmanFilter(dim_x=2, dim_z=2)
    kf.x = np.array([x0[0, 0], x0[1, 0]])
    kf.P = P0
    kf.F = F
    kf.H = H
    kf.Q = Q
    kf.R = R
    estimated_states = []
    for z in zs:
        kf.predict()
        kf.update(z)
        estimated_states.append(kf.x)
    return np.array(estimated_states)


In [56]:
# Position measurements
zs = np.array([[46.8,19.5], [46.72, 19.9]])

# Initial state
x0 = np.array([[45], [17.5]])

# Initial covariance
P0 = np.array([[0.1, 0.0], [0.0, 0.1]])

# State transition matrix
F = np.array([[1.0, 0.1], [0.0, 1.0]])

# Measurement function
H = np.array([[1.0, 0.0], [0.0, 1.0]])

# Process noise covariance
Q = np.array([[0.1, 0.0], [0.0, 0.1]])

# Measurement noise covariance
R = np.array([[0.1, 0.0], [0.0, 0.1]])

# Call the position_estimation function
estimated_states = position_estimation(zs, x0, P0, F, H, Q, R)

print(estimated_states)


[[46.80554324 18.83314856]
 [47.46792024 19.47807517]]
